In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
import sys
import os
if IN_COLAB:
    !git lfs clone https://github.com/sbeeredd04/sandbox_private.git
    !cd /content/sandbox_private/ML/Capstone/
    sys.path.append('/content/sandbox_private/ML/Capstone/')
    #check the current working directory
    print(os.getcwd())
else:
    sys.path.append('..')
    

In [ ]:
import os
# Set this environment for deterministic execution
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [ ]:
if IN_COLAB: 
    #move to /content/sandbox_private/ML/Capstone/
    os.chdir('/content/sandbox_private/ML/Capstone/')
    print(os.getcwd())

In [ ]:
#gpus 
gpu_ids = "1,2"
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids

print("Using GPU IDs: ", gpu_ids)

# Milestone 2: CNN Model for CelebA Multi-Label Classification

## Problem Statement
Upload a person's image and determine different celebrity attributes they have.

## Approach
- Multi-label classification using ResNet18 architecture
- 40 binary attributes predicted simultaneously  
- Custom dataloader for CelebA dataset
- Comprehensive evaluation metrics


## 1. Import Libraries and Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import KFold
import os
import json
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Create output directory for plots
os.makedirs('celeba_plots', exist_ok=True)
print("Setup complete!")


## 2. Load CelebA Dataset and Create DataLoaders

### Design Justification:
- Handles multi-label classification (40 binary attributes)
- Supports data augmentation for training
- Efficient batch processing
- Compatible with PyTorch training pipeline


In [ ]:
# Define image size and batch size
image_size = 128
batch_size = 128
num_workers = 2
data_dir = './data'

# Define training transforms with data augmentation
train_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Define validation/test transforms without augmentation
test_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

print("Transforms defined successfully!")


In [ ]:
#check the current working directory
print(os.getcwd())

# Load CelebA datasets with transforms
print("Loading CelebA datasets...")

try:
    # Try to load with download=True first
    train_dataset = datasets.CelebA(
        root=data_dir,
        split='train',
        transform=train_transform,
        download=True,
        target_type='attr'
    )
    
    val_dataset = datasets.CelebA(
        root=data_dir,
        split='valid',
        transform=test_transform,
        download=True,
        target_type='attr'
    )
    
    test_dataset = datasets.CelebA(
        root=data_dir,
        split='test',
        transform=test_transform,
        download=True,
        target_type='attr'
    )
except:
    # If download fails, try loading from existing files
    print("Download failed, attempting to load from existing files...")
    train_dataset = datasets.CelebA(
        root=data_dir,
        split='train',
        transform=train_transform,
        download=False,
        target_type='attr'
    )
    
    val_dataset = datasets.CelebA(
        root=data_dir,
        split='valid',
        transform=test_transform,
        download=False,
        target_type='attr'
    )
    
    test_dataset = datasets.CelebA(
        root=data_dir,
        split='test',
        transform=test_transform,
        download=False,
        target_type='attr'
    )

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Total images: {len(train_dataset) + len(val_dataset) + len(test_dataset)}")

# Get attribute names
attribute_names = [name for name in train_dataset.attr_names if name.strip()]
num_attributes = len(attribute_names)

print(f"\nNumber of attributes: {num_attributes}")
print(f"Sample attributes: {attribute_names[:5]}...")


In [ ]:
# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f"DataLoaders created with batch size: {batch_size}")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")


## 3. ResNet18 Model Architecture

### Model Selection Justification:

**Why ResNet18 for CelebA?**

1. **Residual Connections**: Skip connections enable training of deeper networks without vanishing gradient issues
2. **Proven Architecture**: ResNet18 has demonstrated excellent performance on face recognition tasks
3. **Appropriate Depth**: 18 layers provide sufficient capacity for 128x128 face images without overfitting
4. **Efficient**: Faster training compared to deeper variants (ResNet50, ResNet101)
5. **Multi-label Adaptation**: Final layer modified for 40 binary classifications with sigmoid activation

**Strengths:**
- Handles vanishing gradients through residual connections
- Good generalization on face datasets
- Efficient training and inference
- Scalable architecture

**Weaknesses:**
- May struggle with highly correlated attributes
- Requires significant computational resources
- Needs careful regularization to prevent overfitting on imbalanced attributes


In [ ]:
class BasicBlock(nn.Module):
    # Basic residual block for ResNet18
    
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                              stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # ReLU activation
        self.relu = nn.ReLU(inplace=True)
        
        # Shortcut connection for dimension matching
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                         stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        # Save identity for residual connection
        identity = x
        
        # First conv block
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        # Second conv block
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Add residual connection: H(x) = F(x) + x
        out += self.shortcut(identity)
        out = self.relu(out)
        
        return out

print("BasicBlock defined!")


In [ ]:
class ResNet18MultiLabel(nn.Module):
    # ResNet18 architecture adapted for multi-label classification
    
    def __init__(self, num_classes=40):
        super(ResNet18MultiLabel, self).__init__()
        
        self.in_channels = 64
        
        # Initial convolution layer for 128x128 images
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers (ResNet-18 configuration: [2, 2, 2, 2])
        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1)   # 32x32 -> 32x32
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)  # 32x32 -> 16x16
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)  # 16x16 -> 8x8
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)  # 8x8 -> 4x4
        
        # Global average pooling and multi-label classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        
        # Note: Sigmoid activation will be applied in BCEWithLogitsLoss
        
        # Initialize weights
        self._initialize_weights()
    
    def _make_layer(self, block, out_channels, blocks, stride=1):
        # Create a residual layer with specified number of blocks
        layers = []
        
        # First block (may downsample)
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        
        # Remaining blocks
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        # Initialize weights using He initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Forward pass through the network
        
        # Initial convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Residual layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Global average pooling and classification
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet18MultiLabel(num_classes=num_attributes)
model = model.to(device)

print(f"ResNet18 Multi-Label model created!")
print(f"Device: {device}")
print(f"Output classes: {num_attributes}")


In [ ]:
# Count model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_params = count_parameters(model)
print(f"Total trainable parameters: {num_params:,}")
print(f"Model size: ~{num_params * 4 / (1024**2):.2f} MB (float32)")


## 4. Training Configuration

### Hyperparameters Justification:

- **Loss Function**: BCEWithLogitsLoss - Combines sigmoid and BCE, numerically stable for multi-label
- **Optimizer**: Adam with lr=0.001 - Adaptive learning rate, good for multi-label problems
- **Weight Decay**: 1e-4 - L2 regularization to prevent overfitting
- **Scheduler**: StepLR - Reduces learning rate every 5 epochs for fine-tuning
- **Batch Size**: 128 - Balanced between GPU memory and convergence speed
- **Epochs**: 20 - Sufficient for convergence with early stopping


In [ ]:
# Training configuration
criterion = nn.BCEWithLogitsLoss()  # For multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

print("Training Configuration:")
print(f"Loss Function: BCEWithLogitsLoss")
print(f"Optimizer: Adam (lr=0.001, weight_decay=1e-4)")
print(f"Scheduler: StepLR (step_size=5, gamma=0.1)")
print(f"Batch Size: {batch_size}")


## 5. Training and Evaluation Functions

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    # Train for one epoch
    model.train()
    running_loss = 0.0
    all_predictions = []
    all_targets = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move data to device
        data = data.to(device)
        target = target.to(device).float()
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        loss = criterion(output, target)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Track metrics
        running_loss += loss.item()
        predictions = torch.sigmoid(output) > 0.5
        all_predictions.append(predictions.cpu())
        all_targets.append(target.cpu())
        
        # Print progress
        print(f'\rEpoch {epoch}: [{batch_idx}/{len(train_loader)}] Loss: {loss.item():.4f}', end='', flush=True)
    
    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader)
    all_predictions = torch.cat(all_predictions).numpy()
    all_targets = torch.cat(all_targets).numpy()
    epoch_acc = accuracy_score(all_targets, all_predictions)
    
    return epoch_loss, epoch_acc

print("train_epoch function defined!")


In [ ]:
def validate_epoch(model, val_loader, criterion, device):
    # Validate for one epoch
    model.eval()
    val_loss = 0.0
    all_predictions = []
    all_targets = []
    all_probs = []
    
    with torch.no_grad():
        for data, target in val_loader:
            # Move data to device
            data = data.to(device)
            target = target.to(device).float()
            
            # Forward pass
            output = model(data)
            loss = criterion(output, target)
            
            # Track metrics
            val_loss += loss.item()
            probs = torch.sigmoid(output)
            predictions = probs > 0.5
            
            all_predictions.append(predictions.cpu())
            all_targets.append(target.cpu())
            all_probs.append(probs.cpu())
    
    # Calculate validation metrics
    val_loss /= len(val_loader)
    all_predictions = torch.cat(all_predictions).numpy()
    all_targets = torch.cat(all_targets).numpy()
    all_probs = torch.cat(all_probs).numpy()
    val_acc = accuracy_score(all_targets, all_predictions)
    
    return val_loss, val_acc, all_predictions, all_targets, all_probs

print("validate_epoch function defined!")


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=20):
    # Main training loop with validation
    
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_acc = 0.0
    
    print("Starting training...")
    print("=" * 80)
    
    for epoch in range(epochs):
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch + 1)
        
        # Validate
        val_loss, val_acc, _, _, _ = validate_epoch(model, val_loader, criterion, device)
        
        # Update learning rate
        scheduler.step()
        
        # Store metrics
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        # Print epoch summary
        print(f'\n', '-' * 80)
        print(f'\nEpoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')
        print(f'  LR: {optimizer.param_groups[0]["lr"]:.6f}')
        print('-' * 80)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_celeba_resnet18.pth')
            print(f'-'*20, 'Best model saved (Val Acc: {val_acc:.4f})', '-'*20)
    
    print("\\nTraining completed!")
    print(f"Best validation accuracy: {best_val_acc:.4f}")
    
    return {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies
    }

print("train_model function defined!")


## 6. Model Training (80/20 Split)

### Training Process:
- Training set: 162,770 images (80%)
- Validation set: 19,867 images (10%)
- Test set: 19,962 images (10%)
- 20 epochs with learning rate scheduling
- Early stopping based on validation accuracy


In [ ]:
# Train the model
history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=20)

## 7. Training Visualization

In [ ]:
def plot_training_curves(history, save_path='celeba_plots/training_curves.png'):
    # Plot training and validation curves
    
    epochs = range(1, len(history['train_losses']) + 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss curves
    ax1.plot(epochs, history['train_losses'], 'b-', label='Training Loss', linewidth=2)
    ax1.plot(epochs, history['val_losses'], 'r-', label='Validation Loss', linewidth=2)
    ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy curves
    ax2.plot(epochs, history['train_accuracies'], 'b-', label='Training Accuracy', linewidth=2)
    ax2.plot(epochs, history['val_accuracies'], 'r-', label='Validation Accuracy', linewidth=2)
    ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print final metrics
    print("="* 50)
    print("Final Training Results:")
    print("="* 50)
    print(f"Final Training Loss: {history['train_losses'][-1]:.4f}")
    print(f"Final Training Accuracy: {history['train_accuracies'][-1]:.4f}")
    print(f"Final Validation Loss: {history['val_losses'][-1]:.4f}")
    print(f"Final Validation Accuracy: {history['val_accuracies'][-1]:.4f}")
    print(f"Best Validation Accuracy: {max(history['val_accuracies']):.4f}")

plot_training_curves(history)


## 8. Model Evaluation

### Evaluation Metrics:
- **Overall Accuracy**: Percentage of correct predictions across all attributes
- **Per-Attribute Accuracy**: Individual accuracy for each of the 40 attributes
- **Precision**: True positives / (True positives + False positives)
- **Recall**: True positives / (True positives + False negatives)
- **F1-Score**: Harmonic mean of precision and recall
- **ROC Curves**: For each attribute to visualize true positive vs false positive rate


In [ ]:
# Load best model for evaluation
model.load_state_dict(torch.load('best_celeba_resnet18.pth'))
print("Best model loaded for evaluation!")


In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
test_loss, test_acc, test_preds, test_targets, test_probs = validate_epoch(
    model, test_loader, criterion, device
)

print("="* 50)
print("Test Set Results:")
print("="* 50)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Total samples: {len(test_targets)}")


In [ ]:
# Calculate per-attribute metrics
def calculate_per_attribute_metrics(predictions, targets, attribute_names):
    # Calculate metrics for each attribute
    
    results = []
    
    for i, attr_name in enumerate(attribute_names):
        attr_preds = predictions[:, i]
        attr_targets = targets[:, i]
        
        # Calculate metrics
        acc = accuracy_score(attr_targets, attr_preds)
        prec = precision_score(attr_targets, attr_preds, zero_division=0)
        rec = recall_score(attr_targets, attr_preds, zero_division=0)
        f1 = f1_score(attr_targets, attr_preds, zero_division=0)
        
        results.append({
            'Attribute': attr_name,
            'Accuracy': acc,
            'Precision': prec,
            'Recall': rec,
            'F1-Score': f1
        })
    
    return pd.DataFrame(results)

# Calculate metrics
metrics_df = calculate_per_attribute_metrics(test_preds, test_targets, attribute_names)

print("\\nPer-Attribute Metrics (Top 10 by F1-Score):")
print(metrics_df.sort_values('F1-Score', ascending=False).head(10))

print("\\nPer-Attribute Metrics (Bottom 10 by F1-Score):")
print(metrics_df.sort_values('F1-Score').head(10))

# Save metrics
metrics_df.to_csv('celeba_per_attribute_metrics.csv', index=False)
print("\\nMetrics saved to celeba_per_attribute_metrics.csv")


In [ ]:
# Visualize per-attribute performance
def plot_attribute_metrics(metrics_df, save_path='celeba_plots/attribute_metrics.png'):
    # Plot metrics for all attributes
    
    fig, axes = plt.subplots(2, 1, figsize=(16, 12))
    
    # Sort by F1-score
    metrics_sorted = metrics_df.sort_values('F1-Score', ascending=False)
    
    # Plot all metrics
    x = np.arange(len(metrics_sorted))
    width = 0.2
    
    axes[0].bar(x - 1.5*width, metrics_sorted['Accuracy'], width, label='Accuracy', alpha=0.8)
    axes[0].bar(x - 0.5*width, metrics_sorted['Precision'], width, label='Precision', alpha=0.8)
    axes[0].bar(x + 0.5*width, metrics_sorted['Recall'], width, label='Recall', alpha=0.8)
    axes[0].bar(x + 1.5*width, metrics_sorted['F1-Score'], width, label='F1-Score', alpha=0.8)
    
    axes[0].set_xlabel('Attributes (sorted by F1-Score)')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Per-Attribute Performance Metrics', fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(metrics_sorted['Attribute'], rotation=90, fontsize=7)
    axes[0].legend()
    axes[0].grid(axis='y', alpha=0.3)
    
    # Plot top 15 attributes by F1-score
    top_15 = metrics_sorted.head(15)
    x_top = np.arange(len(top_15))
    
    axes[1].bar(x_top - 1.5*width, top_15['Accuracy'], width, label='Accuracy', alpha=0.8)
    axes[1].bar(x_top - 0.5*width, top_15['Precision'], width, label='Precision', alpha=0.8)
    axes[1].bar(x_top + 0.5*width, top_15['Recall'], width, label='Recall', alpha=0.8)
    axes[1].bar(x_top + 1.5*width, top_15['F1-Score'], width, label='F1-Score', alpha=0.8)
    
    axes[1].set_xlabel('Top 15 Attributes')
    axes[1].set_ylabel('Score')
    axes[1].set_title('Top 15 Attributes by F1-Score', fontweight='bold')
    axes[1].set_xticks(x_top)
    axes[1].set_xticklabels(top_15['Attribute'], rotation=45, ha='right')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()

plot_attribute_metrics(metrics_df)


In [ ]:
# Plot ROC curves for selected attributes
def plot_roc_curves(targets, probs, attribute_names, selected_attrs=None, save_path='celeba_plots/roc_curves.png'):
    # Plot ROC curves for selected attributes
    
    if selected_attrs is None:
        # Select top 9 attributes by F1-score
        metrics_df_temp = calculate_per_attribute_metrics(
            (probs > 0.5).astype(int), targets, attribute_names
        )
        selected_attrs = metrics_df_temp.sort_values('F1-Score', ascending=False).head(9)['Attribute'].tolist()
    
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.flatten()
    
    for idx, attr_name in enumerate(selected_attrs):
        attr_idx = attribute_names.index(attr_name)
        attr_targets = targets[:, attr_idx]
        attr_probs = probs[:, attr_idx]
        
        # Calculate ROC curve
        fpr, tpr, _ = roc_curve(attr_targets, attr_probs)
        roc_auc = auc(fpr, tpr)
        
        # Plot
        axes[idx].plot(fpr, tpr, 'b-', linewidth=2, label=f'AUC = {roc_auc:.3f}')
        axes[idx].plot([0, 1], [0, 1], 'r--', linewidth=1, label='Random')
        axes[idx].set_xlabel('False Positive Rate')
        axes[idx].set_ylabel('True Positive Rate')
        axes[idx].set_title(f'{attr_name}', fontweight='bold')
        axes[idx].legend(loc='lower right')
        axes[idx].grid(True, alpha=0.3)
    
    plt.suptitle('ROC Curves for Top 9 Attributes', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()

plot_roc_curves(test_targets, test_probs, attribute_names)


In [ ]:
# Confusion matrices for selected attributes
def plot_confusion_matrices(predictions, targets, attribute_names, selected_attrs=None, save_path='celeba_plots/confusion_matrices.png'):
    # Plot confusion matrices for selected attributes
    
    if selected_attrs is None:
        # Select 6 diverse attributes
        selected_attrs = ['Male', 'Smiling', 'Eyeglasses', 'Young', 'Wearing_Hat', 'Attractive']
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, attr_name in enumerate(selected_attrs):
        attr_idx = attribute_names.index(attr_name)
        attr_preds = predictions[:, attr_idx]
        attr_targets = targets[:, attr_idx]
        
        # Calculate confusion matrix
        cm = confusion_matrix(attr_targets, attr_preds)
        
        # Plot
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                   xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
        axes[idx].set_title(attr_name, fontweight='bold')
        axes[idx].set_xlabel('Predicted')
        axes[idx].set_ylabel('Actual')
    
    plt.suptitle('Confusion Matrices for Selected Attributes', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()

plot_confusion_matrices(test_preds, test_targets, attribute_names)


## 9. Cross-Validation (K-Fold)

### Cross-Validation Strategy:
- 3-Fold cross-validation on training set
- Each fold trains a separate model
- Averages performance across folds
- Provides robust performance estimate
- Helps detect overfitting


In [ ]:
def cross_validate_model(train_dataset, device, num_folds=3, epochs=10, batch_size=128):
    # Perform k-fold cross-validation
    
    print(f"Starting {num_folds}-Fold Cross-Validation...")
    print("=" * 80)
    
    # Create indices for k-fold
    dataset_size = len(train_dataset)
    indices = list(range(dataset_size))
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
        print(f"\\nFold {fold + 1}/{num_folds}")
        print("-" * 80)
        
        # Create data loaders for this fold
        train_subset = Subset(train_dataset, train_idx)
        val_subset = Subset(train_dataset, val_idx)
        
        fold_train_loader = DataLoader(train_subset, batch_size=batch_size, 
                                      shuffle=True, num_workers=2, pin_memory=True)
        fold_val_loader = DataLoader(val_subset, batch_size=batch_size, 
                                    shuffle=False, num_workers=2, pin_memory=True)
        
        print(f"Train samples: {len(train_subset)}")
        print(f"Val samples: {len(val_subset)}")
        
        # Create new model for this fold
        fold_model = ResNet18MultiLabel(num_classes=num_attributes).to(device)
        fold_criterion = nn.BCEWithLogitsLoss()
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=0.001, weight_decay=1e-4)
        fold_scheduler = optim.lr_scheduler.StepLR(fold_optimizer, step_size=3, gamma=0.1)
        
        # Train for this fold
        best_val_acc = 0.0
        
        for epoch in range(epochs):
            # Train
            train_loss, train_acc = train_epoch(fold_model, fold_train_loader, 
                                               fold_criterion, fold_optimizer, device, epoch + 1)
            
            # Validate
            val_loss, val_acc, _, _, _ = validate_epoch(fold_model, fold_val_loader, 
                                                        fold_criterion, device)
            
            fold_scheduler.step()
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
            
            print(f'Epoch {epoch+1}/{epochs}: Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}')
        
        fold_results.append({
            'fold': fold + 1,
            'best_val_acc': best_val_acc,
            'final_train_acc': train_acc,
            'final_val_acc': val_acc
        })
        
        print(f"Fold {fold + 1} Best Val Accuracy: {best_val_acc:.4f}")
    
    # Summary
    results_df = pd.DataFrame(fold_results)
    
    print("\\n" + "=" * 80)
    print("Cross-Validation Results Summary")
    print("=" * 80)
    print(results_df)
    print(f"\\nMean Best Val Accuracy: {results_df['best_val_acc'].mean():.4f} ± {results_df['best_val_acc'].std():.4f}")
    print(f"Mean Final Val Accuracy: {results_df['final_val_acc'].mean():.4f} ± {results_df['final_val_acc'].std():.4f}")
    
    return results_df

print("cross_validate_model function defined!")


In [ ]:
# Perform cross-validation (using subset for speed)
# Note: Using first 30000 samples for demonstration; use full dataset for production
cv_subset = Subset(train_dataset, range(30000))
cv_results = cross_validate_model(cv_subset, device, num_folds=3, epochs=5, batch_size=128)


## 10. Model Comparison Summary


In [ ]:
# Print final summary statistics
print("="* 80)
print("FINAL MODEL PERFORMANCE SUMMARY")
print("="* 80)
print(f"\\nDataset Statistics:")
print(f"  Training samples: {len(train_dataset)}")
print(f"  Validation samples: {len(val_dataset)}")
print(f"  Test samples: {len(test_dataset)}")
print(f"  Total attributes: {num_attributes}")

print(f"\\nModel Architecture:")
print(f"  Model: ResNet18 (Multi-Label)")
print(f"  Parameters: {count_parameters(model):,}")
print(f"  Input size: 128x128x3")
print(f"  Output size: {num_attributes}")

print(f"\\nTraining Configuration:")
print(f"  Loss: BCEWithLogitsLoss")
print(f"  Optimizer: Adam (lr=0.001, weight_decay=1e-4)")
print(f"  Batch size: {batch_size}")
print(f"  Epochs trained: {len(history['train_losses'])}")

print(f"\\nPerformance Metrics:")
print(f"  Test Accuracy: {test_acc:.4f}")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Mean Attribute F1-Score: {metrics_df['F1-Score'].mean():.4f}")
print(f"  Best Attribute F1-Score: {metrics_df['F1-Score'].max():.4f}")
print(f"  Worst Attribute F1-Score: {metrics_df['F1-Score'].min():.4f}")

print(f"\\nTop 5 Performing Attributes:")
top_5_attrs = metrics_df.nlargest(5, 'F1-Score')
for _, row in top_5_attrs.iterrows():
    print(f"  {row['Attribute']}: F1={row['F1-Score']:.4f}, Acc={row['Accuracy']:.4f}")

print(f"\\nBottom 5 Performing Attributes:")
bottom_5_attrs = metrics_df.nsmallest(5, 'F1-Score')
for _, row in bottom_5_attrs.iterrows():
    print(f"  {row['Attribute']}: F1={row['F1-Score']:.4f}, Acc={row['Accuracy']:.4f}")

print("\\n" + "="* 80)
print("Milestone 2 Complete!")
print("="* 80)


In [ ]:
# Save final results
final_results = {
    'test_accuracy': float(test_acc),
    'test_loss': float(test_loss),
    'mean_f1_score': float(metrics_df['F1-Score'].mean()),
    'best_validation_accuracy': float(max(history['val_accuracies'])),
    'training_epochs': len(history['train_losses'])
}

# Save results to file
with open('celeba_final_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("Results saved to celeba_final_results.json")
print("\\nAll files saved:")
print("  - best_celeba_resnet18.pth (model weights)")
print("  - celeba_per_attribute_metrics.csv (detailed metrics)")
print("  - celeba_final_results.json (summary results)")
print("  - celeba_plots/ (all visualizations)")
